 # Tugas Presentasi Kelompok 8

# 1.Klasifikasi

 # Melakukan labelling pada mysql berdasarkan avgtone lebih dari 0 sama dengan positif lainya negatif
![title](gambar/picture.png)

# Contoh Hasil Data
![title](gambar/picture1.png)

# Melakukan export ke dalam format CSV(sentimen.csv) dan melakukan preprocessing teks isi berita dengan menggunakan scapy

In [2]:
import pandas as pd
header = ['berita','sourceurl','label']
df=pd.read_csv('sentimen.csv',names=header) #--> dataframe
df_berita = df['berita'] #--> dataframe berita
df_berita.head()

0    President Trump said on Wednesday after a meet...
1    Investors and economists for months have clung...
2    The ongoing trade spat between the U.S. and Ch...
3    Export earnings’ growth in September surged by...
4    ×remaining ofThank you for Reading!On your nex...
Name: berita, dtype: object

In [3]:
df_berita[0]

'President Trump said on Wednesday after a meeting with European Commission President Jean-Claude Juncker that they had agreed to work on lowering trade barriers.Westwood One reports the president says that includes steel and aluminum tariffs.President Trump says the United States and the European Union have agreed to work toward “zero tariffs” and “zero subsidies” on non-automobile goods.Trump also says the EU has also agreed to buy “a lot of soybeans” and increase its imports of liquefied natural gas from the U.S. Trump says the EU will become a “massive buyer” of LNG to help diversify their energy supply.The president and the EU’s chief executive made statements at a joint event today following the meeting on trade.'

In [4]:
import spacy
import string
result_dict={'berita':[]}
nlp = spacy.load('en')
size = len(df_berita)
exclude = set(string.punctuation)
for num in range(size):
    doc = nlp(df_berita[num])
    remove=['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE','NUM','SYM']
    result = [token.lemma_ for token in doc if token.pos_ not in remove and token.is_stop is not True]
    sentence = ' '.join(result)
    sentence = ''.join(char for char in sentence if char not in exclude)
    result_dict['berita'].append(sentence)
df_clean = pd.DataFrame.from_dict(result_dict) #dataframe clean text

In [5]:
df_clean['berita'][0]

'president trump say wednesday meeting european commission president jean claude juncker agree work lower trade barrier westwood one report president say include steel aluminum tariff president trump say united states european union agree work tariff subsidy non automobile good trump say eu agree buy lot soybean increase import liquefy natural gas us trump say eu massive buyer lng help diversify energy supply president eu chief executive statement joint event today follow meeting trade'

# Melakukan Vektorisasi dengan menggunakan Bag of word  dan tf-idf sklearn

In [6]:
corpus = [sentence for sentence in df_clean['berita']]

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_bow = CountVectorizer(max_df=0.8,analyzer="word",max_features=5000)
vectorizer_bow.fit(corpus)
vector_bow = vectorizer_bow.transform(corpus)
print(vector_bow.shape)

(800, 5000)


In [8]:
vector_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer(max_df=0.8,analyzer="word",use_idf=True,max_features=5000)
vectorizer_tfidf.fit(corpus)
vector_tfidf = vectorizer_tfidf.transform(corpus)
print(vector_tfidf.shape)

(800, 5000)


In [10]:
vector_tfidf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Training menggunakan Multinomial Naive sklearn dan Evaluasi 10 kfold

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
import statistics
label = [label for label in df["label"]]
mnb = MultinomialNB()
accuracy = cross_validate(mnb,vector_bow,label,cv=10,scoring='accuracy',return_train_score=False)
print('Bow dan MNB accuracy:')
print(accuracy['test_score'])
print()
precision = cross_validate(mnb,vector_bow,label,cv=10,scoring='precision',return_train_score=False)
print('Bow dan MNB precision:')
print(precision['test_score'])
print()
recall = cross_validate(mnb,vector_bow,label,cv=10,scoring='recall',return_train_score=False)
print('Bow dan MNB recall:')
print(recall['test_score'])
print("----------------------------------")
print("Rata-Rata akurasi : "+ str(statistics.mean(accuracy['test_score'])))
print("Rata-Rata precision : "+ str(statistics.mean(precision['test_score'])))
print("Rata-Rata recall : "+ str(statistics.mean(recall['test_score'])))
print("----------------------------------")
print()
accuracy = cross_validate(mnb,vector_tfidf,label,cv=10,scoring='accuracy',return_train_score=False)
print('Tfidf dan MNB accuracy:')
print(accuracy['test_score'])
print()
precision = cross_validate(mnb,vector_tfidf,label,cv=10,scoring='precision',return_train_score=False)
print('Tfidf dan MNB precision:')
print(precision['test_score'])
print()
recall = cross_validate(mnb,vector_tfidf,label,cv=10,scoring='recall',return_train_score=False)
print('Tfidf dan MNB recall:')
print(recall['test_score'])
print("----------------------------------")
print("Rata-Rata akurasi : "+ str(statistics.mean(accuracy['test_score'])))
print("Rata-Rata precision : "+ str(statistics.mean(precision['test_score'])))
print("Rata-Rata recall : "+ str(statistics.mean(recall['test_score'])))
print("----------------------------------")
print()

Bow dan MNB accuracy:
[0.6375 0.5875 0.7375 0.7125 0.675  0.7    0.725  0.775  0.7375 0.75  ]

Bow dan MNB precision:
[0.67741935 0.58536585 0.73170732 0.71794872 0.71875    0.7
 0.70454545 0.78947368 0.77142857 0.8125    ]

Bow dan MNB recall:
[0.525 0.6   0.75  0.7   0.575 0.7   0.775 0.75  0.675 0.65 ]
----------------------------------
Rata-Rata akurasi : 0.70375
Rata-Rata precision : 0.7209138953703688
Rata-Rata recall : 0.67
----------------------------------

Tfidf dan MNB accuracy:
[0.65   0.625  0.8    0.675  0.675  0.6875 0.75   0.8    0.7125 0.7375]

Tfidf dan MNB precision:
[0.71428571 0.63888889 0.83333333 0.70588235 0.73333333 0.7027027
 0.75       0.85294118 0.77419355 0.82758621]

Tfidf dan MNB recall:
[0.5   0.575 0.75  0.6   0.55  0.65  0.75  0.725 0.6   0.6  ]
----------------------------------
Rata-Rata akurasi : 0.71125
Rata-Rata precision : 0.7533147257239385
Rata-Rata recall : 0.63
----------------------------------



# MNB Melakukan prediksi pada data bersih diluar data training

In [12]:
mnb.fit(vector_bow,label)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
print("Predicting.....")
df_test = pd.read_csv('cleandata.csv')
df_test_berita = [df_test['berita'][index] for index in range(5)]
new_vector_mnb = vectorizer_bow.transform(df_test_berita)
print("Result.....")
predicted = mnb.predict(new_vector_mnb)
for doc,label in zip(df_test_berita,predicted):
    print(str(doc) + " : " + str(label))
    print()

Predicting.....
Result.....
billionaire warren buffett say expect blow trade war develop united states china country benefit trade analyst list hard line demand trump administration hand china week difficult resolve trade conflict world large economy separate issue buffett say wells fargo clean act misconduct bank recent year berkshire hathaway own percent wells fargo stock buffett say saturday plan sell buffett say wells fargo big mistake wrong incentive place employee make stride : -1

president trump say wednesday meeting european commission president jean claude juncker agree work lower trade barrier westwood one report president say include steel aluminum tariff president trump say united states european union agree work tariff subsidy non automobile good trump say eu agree buy lot soybean increase import liquefy natural gas us trump say eu massive buyer lng help diversify energy supply president eu chief executive statement joint event today follow meeting trade : 1

ongoing trad

# Evaluasi dengan confusion matrix pada data training

In [14]:
from sklearn.metrics import confusion_matrix, classification_report
# suppose your predictions are stored in a variable called preds
# and the true values are stored in a variable called y
prediksi = mnb.predict(vector_bow)
label = [label for label in df["label"]]
#confusion_matrix(y_true, y_pred, labels=["True", "False"])
print(confusion_matrix(label, prediksi))
print(classification_report(label, prediksi))

[[386  14]
 [ 24 376]]
              precision    recall  f1-score   support

          -1       0.94      0.96      0.95       400
           1       0.96      0.94      0.95       400

   micro avg       0.95      0.95      0.95       800
   macro avg       0.95      0.95      0.95       800
weighted avg       0.95      0.95      0.95       800



# Training menggunakan Random Forest sklearn dan Evaluasi 10 kfold

In [15]:
from sklearn.ensemble import RandomForestClassifier
import statistics
label = [label for label in df["label"]]
trees = [50,75,100]
for tree in trees:
    print("Jumlah Tree: "+ str(tree))
    print()
    rf = RandomForestClassifier(n_estimators=tree,random_state=0,criterion='gini',max_features='auto')
    accuracy = cross_validate(rf,vector_bow,label,cv=10,scoring='accuracy',return_train_score=False)
    print('Bow dan RF accuracy:')
    print(accuracy['test_score'])
    print()
    precision = cross_validate(rf,vector_bow,label,cv=10,scoring='precision',return_train_score=False)
    print('Bow dan RF precision:')
    print(precision['test_score'])
    print()
    recall = cross_validate(rf,vector_bow,label,cv=10,scoring='recall',return_train_score=False)
    print('Bow dan RF recall:')
    print(recall['test_score'])
    print("----------------------------------")
    print("Rata-Rata akurasi : "+ str(statistics.mean(accuracy['test_score'])))
    print("Rata-Rata precision : "+ str(statistics.mean(precision['test_score'])))
    print("Rata-Rata recall : "+ str(statistics.mean(recall['test_score'])))
    print("----------------------------------")
    print()
    accuracy = cross_validate(rf,vector_tfidf,label,cv=10,scoring='accuracy',return_train_score=False)
    print('Tfidf dan RF accuracy:')
    print(accuracy['test_score'])
    print()
    precision = cross_validate(rf,vector_tfidf,label,cv=10,scoring='precision',return_train_score=False)
    print('Tfidf dan RF precision:')
    print(precision['test_score'])
    print()
    recall = cross_validate(rf,vector_tfidf,label,cv=10,scoring='recall',return_train_score=False)
    print('Tfidf dan RF recall:')
    print(recall['test_score'])
    print("----------------------------------")
    print("Rata-Rata akurasi : "+ str(statistics.mean(accuracy['test_score'])))
    print("Rata-Rata precision : "+ str(statistics.mean(precision['test_score'])))
    print("Rata-Rata recall : "+ str(statistics.mean(recall['test_score'])))
    print("----------------------------------")
    print()

Jumlah Tree: 50

Bow dan RF accuracy:
[0.6375 0.7    0.65   0.6375 0.6375 0.65   0.65   0.6875 0.675  0.725 ]

Bow dan RF precision:
[0.63414634 0.66       0.64285714 0.62222222 0.64102564 0.67647059
 0.66666667 0.7027027  0.675      0.78125   ]

Bow dan RF recall:
[0.65  0.825 0.675 0.7   0.625 0.575 0.6   0.65  0.675 0.625]
----------------------------------
Rata-Rata akurasi : 0.665
Rata-Rata precision : 0.6702341305173084
Rata-Rata recall : 0.66
----------------------------------

Tfidf dan RF accuracy:
[0.575  0.675  0.6625 0.625  0.6625 0.65   0.675  0.725  0.6375 0.7375]

Tfidf dan RF precision:
[0.59375    0.675      0.64444444 0.61363636 0.6969697  0.65789474
 0.675      0.73684211 0.61702128 0.74358974]

Tfidf dan RF recall:
[0.475 0.675 0.725 0.675 0.575 0.625 0.675 0.7   0.725 0.725]
----------------------------------
Rata-Rata akurasi : 0.6625
Rata-Rata precision : 0.6654148367341257
Rata-Rata recall : 0.6575
----------------------------------

Jumlah Tree: 75

Bow dan RF 

# RF Melakukan prediksi pada data bersih diluar data training

In [16]:
rf = RandomForestClassifier(n_estimators=100,random_state=0,criterion='gini',max_features='auto')
rf.fit(vector_bow,label)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [17]:
print("Predicting.....")
new_vector_rf = vectorizer_bow.transform(df_test_berita)
print("Result.....")
prediksi = rf.predict(new_vector_rf)
for doc,label in zip(df_test_berita,prediksi):
    print(str(doc) + " : " + str(label))
    print()

Predicting.....
Result.....
billionaire warren buffett say expect blow trade war develop united states china country benefit trade analyst list hard line demand trump administration hand china week difficult resolve trade conflict world large economy separate issue buffett say wells fargo clean act misconduct bank recent year berkshire hathaway own percent wells fargo stock buffett say saturday plan sell buffett say wells fargo big mistake wrong incentive place employee make stride : -1

president trump say wednesday meeting european commission president jean claude juncker agree work lower trade barrier westwood one report president say include steel aluminum tariff president trump say united states european union agree work tariff subsidy non automobile good trump say eu agree buy lot soybean increase import liquefy natural gas us trump say eu massive buyer lng help diversify energy supply president eu chief executive statement joint event today follow meeting trade : 1

ongoing trad

# Evaluasi dengan confusion matrix pada data training

In [18]:
# suppose your predictions are stored in a variable called preds
# and the true values are stored in a variable called y
prediksi = rf.predict(vector_bow)
label = [label for label in df["label"]]
#confusion_matrix(y_true, y_pred, labels=["True", "False"])
print(confusion_matrix(label, prediksi))
print(classification_report(label, prediksi))

[[398   2]
 [  2 398]]
              precision    recall  f1-score   support

          -1       0.99      0.99      0.99       400
           1       0.99      0.99      0.99       400

   micro avg       0.99      0.99      0.99       800
   macro avg       0.99      0.99      0.99       800
weighted avg       0.99      0.99      0.99       800



# Hasil Visualisasi/ Averaging dengan Menggunakan Tableau dari data hasil dan USDCHY Forex


In [29]:
%%html
<img src='gambar/picture2.png', width=800, height=800>

In [31]:
%%html
<img src='gambar/picture3.png', width=800, height=800>